In [17]:
import pandas as pd

df = pd.read_csv('bike.csv')
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0.0,0,1,9.84,14.395,81,0.0000,3,13,NaN
1,2011-01-01 01:00:00,1,0.0,0,1,9.02,13.635,80,0.0000,8,32,NaN
2,2011-01-01 02:00:00,1,0.0,0,1,9.02,13.635,80,0.0000,5,27,NaN
3,2011-01-01 03:00:00,1,0.0,0,1,9.84,14.395,75,0.0000,3,10,13.0
4,2011-01-01 04:00:00,1,0.0,0,1,9.84,14.395,75,0.0000,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0.0,1,1,15.58,19.695,50,26.0027,7,329,336.0
10882,2012-12-19 20:00:00,4,0.0,1,1,14.76,17.425,57,15.0013,10,231,241.0
10883,2012-12-19 21:00:00,4,0.0,1,1,13.94,15.910,61,15.0013,4,164,168.0
10884,2012-12-19 22:00:00,4,0.0,1,1,13.94,17.425,61,6.0032,12,117,129.0


In [18]:
df.iloc[0,0]

'2011-01-01 00:00:00'

In [22]:
df.loc[0,'datetime']

'2011-01-01 00:00:00'

In [25]:
df.iloc[:3, 1:3].sum()

season     3.0
holiday    0.0
dtype: float64

In [34]:
df.iloc[:, [1,9]].groupby('season').mean()

,casual
season,
1,15.489576
2,47.446762
3,52.220271
4,28.580834


In [38]:
df.iloc[:, [1,9]].groupby('season').agg(['sum', 'min', 'max'])

casual         
           sum min  max
season                 
1        41605   0  367
2       129672   0  361
3       142718   0  350
4        78140   0  362

In [41]:
df_sub = df[(df.season == 1) & (df.holiday == 1)]
df_sub

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
372,2011-01-17 00:00:00,1,1.0,0,2,8.20,9.850,47,15.0013,1,16,17.0
373,2011-01-17 01:00:00,1,1.0,0,2,8.20,9.850,44,12.9980,1,15,16.0
374,2011-01-17 02:00:00,1,1.0,0,2,7.38,8.335,43,16.9979,0,8,8.0
375,2011-01-17 03:00:00,1,1.0,0,2,7.38,9.090,43,12.9980,0,2,2.0
376,2011-01-17 04:00:00,1,1.0,0,2,7.38,9.850,43,8.9981,1,2,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5799,2012-01-16 19:00:00,1,1.0,0,1,10.66,12.120,56,19.0012,10,134,144.0
5800,2012-01-16 20:00:00,1,1.0,0,1,10.66,12.120,60,19.0012,2,88,90.0
5801,2012-01-16 21:00:00,1,1.0,0,1,11.48,12.880,52,22.0028,3,46,49.0
5802,2012-01-16 22:00:00,1,1.0,0,2,12.30,13.635,49,23.9994,4,39,43.0


In [48]:
# causal 값이 10 이상인 데이터의 season별 registered의 합계

df[df['casual']>=10][['season', 'registered']].groupby('season').sum()

,registered
season,
1,170649
2,434124
3,476165
4,393582


In [51]:
df.isnull().sum()

datetime      0
season        0
holiday       3
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         3
dtype: int64

In [55]:
df[['count']].fillna(df['count'].mean())

,count
0,191.618855
1,191.618855
2,191.618855
3,13.000000
4,1.000000
...,...
10881,336.000000
10882,241.000000
10883,168.000000
10884,129.000000


In [59]:
# 1%의 데이터 삭제
df_sub = df.loc[df['casual'] < df['casual'].quantile(0.99), ['casual']]
df_sub

,casual
0,3
1,8
2,5
3,3
4,0
...,...
10881,7
10882,10
10883,4
10884,12


In [58]:
df['casual'].quantile(0.99)

240.14999999999964

In [67]:
# db에서 판다스로 불러오기
import pymysql
import pandas as pd

# DB 접속
db = pymysql.connect(host="localhost", port=3306, user="root", password="peter2012!", db="sba")
cursor = db.cursor()

# 가수명, 팔로워수, 노래 제목, 앨범 명 DB에서 불러오기
sql = """
SELECT singer.name, singer.follower, song.title, song.album
FROM singer 
    JOIN song 
    ON singer.id = song.singer_id;
"""
cursor.execute(sql)
rows = cursor.fetchall()
db.close()

# 데이터를 dataframe으로 저장
df = pd.DataFrame(rows)
df.columns = ['name', 'follower', 'title', 'album']
df

,name,follower,title,album
0,NewJeans,8.7M,Super Shy,"NewJeans 2nd EP ""Get Up"""
1,정국,0,Seven (feat. Latto) - Clean Ver.,Seven (feat. Latto) - Clean Ver.
2,NewJeans,8.7M,ETA,"NewJeans 2nd EP ""Get Up"""
3,(여자)아이들,11.1M,퀸카 (Queencard),I feel
4,IVE (아이브),3.9M,I AM,"I""ve IVE"
...,...,...,...,...
95,NCT DREAM,12.8M,SOS,ISTJ - The 3rd Album
96,이채연,2.4M,KNOCK,Over The Moon
97,NCT DREAM,12.8M,Pretzel (♡),ISTJ - The 3rd Album
98,NCT DREAM,12.8M,제자리 걸음 (Starry Night),ISTJ - The 3rd Album


In [72]:
# 가수별로 TOP 100에 있는 제목의 수를 출력 하여 정렬
df_sub = df[['album', 'title']].groupby('album').count().sort_values(by='title', ascending=False)
df_sub.to_csv("result.csv")

In [73]:
import numpy as np
np.zeros([5,4])

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [74]:
df = pd.DataFrame(np.zeros([5,4]))
df

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [76]:
a = []
for i in range(100):
    a.append(i)
[x for x in range(100)]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]